In [1]:
import numpy as np 
import time 
from useful_function_joint import *
import pymanopt
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem
from typing import Sequence
from scipy.spatial.transform import Rotation as R

import networkx as nx

In [2]:
scale_shift = 50

In [3]:
e1 = np.array((1,0,0))*scale_shift
e2 = np.array((0,1,0))*scale_shift
e3 = np.array((0,0,1))*scale_shift
e0 = np.array([0,0,0])

In [4]:
t1 = np.array((25,0,0))
t2 = np.array((560,0,0))
t3 = np.array((515/2,0,25))
t4 = np.array((515/2,0,0))
t5 = np.array((90,0,0))

In [5]:
joints_ref = np.array([0,-90,90,0,0,0])*np.pi/180

In [6]:
R1 = Rz(joints_ref[0])
R2 = Ry(joints_ref[1])
R3 = Ry(joints_ref[2])
R4 = Rx(joints_ref[3])
R5 = Ry(joints_ref[4])
R6 = Rx(joints_ref[5])

In [7]:
T1 = T(t1)
T2 = T(t2)
T3 = T(t3)
T4 = T(t4)
T5 = T(t5)

In [8]:
axes = ['z','y','y','x','y','x']
translation = [T(e0),T1,T2,T3,T4,T5]
rotation = [R1,R2,R3,R4,R5,R6]

step_vect = {'z': [e0,e3],
             'y': [e2,-e2],
             'x': [e0,-e1]}

In [9]:
position = pose_robot_(joints_ref,'camera')[:3,3]
#np.array([400,0,695])
direction = np.array([1,0,0.])#/np.sqrt(2)

G = nx.Graph()

G.add_node(1,pos_ref=e1)
G.add_node(2,pos_ref=e2)
G.add_edge(1,2, weight  = np.linalg.norm(e1-e2))


count = 2

for i, ax in enumerate(axes):
    v1,v2 = step_vect[ax]

    transform = []

    for j in range(i+1):
        transform.append(translation[j])
        transform.append(rotation[j])
    transform = np.array(transform)
    transformation = np.linalg.multi_dot(transform)

    G.add_node(count+1,pos_ref=(transformation@T(v2))[:3,3])
    G.add_node(count+2,pos_ref=(transformation@T(v1))[:3,3])

    node_pos_ref = nx.get_node_attributes(G,'pos_ref')

    G.add_edge(count+2,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count+2]))

    G.add_edge(count,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
    G.add_edge(count-1,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))

    G.add_edge(count,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
    G.add_edge(count-1,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))

    count = count + 2

mapping = {1: "x", 2: "y", 3: "a1_plus", 4: "a1_origin", 5: "a2_minus", 6: " a2_plus", 7: "a3_minus", 8:"a3_plus", 
           9: "a4_minus", 10:"a4_origin", 11: "a5_minus", 12:"a5_plus", 13: "a6_minus", 14:"a6_origin"}
G = nx.relabel_nodes(G, mapping)

#G.add_edge('a1_origin','a6_origin', weight = np.linalg.norm(position))

G.add_edge('x','a6_origin', weight = np.linalg.norm(position-e1))
G.add_edge('y','a6_origin', weight = np.linalg.norm(position-e2))
G.add_edge('a1_plus','a6_origin', weight = np.linalg.norm(position-e3))

  
G.add_node('direction',pos_ref=(transformation@T(e1/scale_shift))[:3,3])

G.add_edge('a6_origin','direction', weight = np.linalg.norm(direction))

G.add_edge('x','direction', weight = np.linalg.norm(position+direction-e1))
G.add_edge('y','direction', weight = np.linalg.norm(position+direction-e2))
G.add_edge('a1_plus','direction', weight = np.linalg.norm(position+direction-e3))
  

In [10]:
#G.add_edge('a1_origin','a6_minus', weight = np.linalg.norm(position-scale_shift*direction))

G.add_edge('x','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e1))
G.add_edge('y','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e2))
G.add_edge('a1_plus','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e3))



In [11]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_plus'), ('x', 'a1_origin'), ('x', 'a6_origin'), ('x', 'direction'), ('x', 'a6_minus'), ('y', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a6_origin'), ('y', 'direction'), ('y', 'a6_minus'), ('a1_plus', 'a1_origin'), ('a1_plus', 'a2_minus'), ('a1_plus', ' a2_plus'), ('a1_plus', 'a6_origin'), ('a1_plus', 'direction'), ('a1_plus', 'a6_minus'), ('a1_origin', 'a2_minus'), ('a1_origin', ' a2_plus'), ('a2_minus', ' a2_plus'), ('a2_minus', 'a3_minus'), ('a2_minus', 'a3_plus'), (' a2_plus', 'a3_minus'), (' a2_plus', 'a3_plus'), ('a3_minus', 'a3_plus'), ('a3_minus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_plus', 'a4_origin'), ('a4_minus', 'a4_origin'), ('a4_minus', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_origin', 'a5_plus'), ('a5_minus', 'a5_plus'), ('a5_minus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_plus', 'a6_origin'), ('a6_minus', 'a6_origin'), ('a6_origin', 'direction')])

In [12]:
nx.get_node_attributes(G,'pos_ref')

{'x': array([50,  0,  0]),
 'y': array([ 0, 50,  0]),
 'a1_plus': array([ 0.,  0., 50.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([ 25., -50.,   0.]),
 ' a2_plus': array([25., 50.,  0.]),
 'a3_minus': array([ 25., -50., 560.]),
 'a3_plus': array([ 25.,  50., 560.]),
 'a4_minus': array([232.5,   0. , 585. ]),
 'a4_origin': array([282.5,   0. , 585. ]),
 'a5_minus': array([540., -50., 585.]),
 'a5_plus': array([540.,  50., 585.]),
 'a6_minus': array([580.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.]),
 'direction': array([631.,   0., 585.])}

In [13]:
Y_init = np.zeros((3,43))

for i, e in enumerate(list(G.edges)):
    print(e[0])
    p1 = G.nodes[e[0]]['pos_ref']
    p2 = G.nodes[e[1]]['pos_ref']
    Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

Y_init.shape

x
x
x
x
x
x
y
y
y
y
y
a1_plus
a1_plus
a1_plus
a1_plus
a1_plus
a1_plus
a1_origin
a1_origin
a2_minus
a2_minus
a2_minus
 a2_plus
 a2_plus
a3_minus
a3_minus
a3_minus
a3_plus
a3_plus
a4_minus
a4_minus
a4_minus
a4_origin
a4_origin
a5_minus
a5_minus
a5_minus
a5_plus
a5_plus
a6_minus
a6_origin


(3, 43)

In [14]:
cost(Y_init)

NameError: name 'cost' is not defined

In [15]:
nx.get_node_attributes(G,'pos_ref')


{'x': array([50,  0,  0]),
 'y': array([ 0, 50,  0]),
 'a1_plus': array([ 0.,  0., 50.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([ 25., -50.,   0.]),
 ' a2_plus': array([25., 50.,  0.]),
 'a3_minus': array([ 25., -50., 560.]),
 'a3_plus': array([ 25.,  50., 560.]),
 'a4_minus': array([232.5,   0. , 585. ]),
 'a4_origin': array([282.5,   0. , 585. ]),
 'a5_minus': array([540., -50., 585.]),
 'a5_plus': array([540.,  50., 585.]),
 'a6_minus': array([580.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.]),
 'direction': array([631.,   0., 585.])}

In [16]:
G.edges()
nx.get_edge_attributes(G, "weight")

{('x', 'y'): 70.71067811865476,
 ('x', 'a1_plus'): 70.71067811865476,
 ('x', 'a1_origin'): 50.0,
 ('x', 'a6_origin'): 823.786987030021,
 ('x', 'direction'): 824.4913583537428,
 ('x', 'a6_minus'): 789.3826701923472,
 ('y', 'a1_plus'): 70.71067811865476,
 ('y', 'a1_origin'): 50.0,
 ('y', 'a6_origin'): 861.1765208132418,
 ('y', 'direction'): 861.9083477957503,
 ('y', 'a6_minus'): 825.3029746705146,
 ('a1_plus', 'a1_origin'): 50.0,
 ('a1_plus', 'a2_minus'): 75.0,
 ('a1_plus', ' a2_plus'): 75.0,
 ('a1_plus', 'a6_origin'): 826.5137627408269,
 ('a1_plus', 'direction'): 827.2762537387375,
 ('a1_plus', 'a6_minus'): 789.0659034580065,
 ('a1_origin', 'a2_minus'): 55.90169943749474,
 ('a1_origin', ' a2_plus'): 55.90169943749474,
 ('a2_minus', ' a2_plus'): 100.0,
 ('a2_minus', 'a3_minus'): 560.0,
 ('a2_minus', 'a3_plus'): 568.8585061331157,
 (' a2_plus', 'a3_minus'): 568.8585061331157,
 (' a2_plus', 'a3_plus'): 560.0,
 ('a3_minus', 'a3_plus'): 100.0,
 ('a3_minus', 'a4_minus'): 214.89823172841608,
 

In [59]:

pos_ref = nx.get_node_attributes(G,'pos_ref')

weight = nx.get_edge_attributes(G, "weight")
D = np.diag([weight[i] for i in weight.keys()])
C = incidence_matrix_(G,oriented=True).toarray()
C = np.array(C)

d = 3
n = D.shape[0]

manifold_ = manifolds.PSDFixedRank(15,3)

W = np.eye(n)

Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D;
Q1 = D@W@D;

@pymanopt.function.numpy(manifold_)
def cost(X):
    return np.sum((np.linalg.norm(C.transpose()@X,'fro')**2-D**2)**2)

@pymanopt.function.numpy(manifold_)
def euclidean_gradient(X):
    print(X.shape)
    print((C@C.transpose()@X).shape)
    print((4*C@(np.linalg.norm(C.transpose()@X,axis=1)**2-D**2)@C.transpose()@X).shape)
    return 4*C@C.transpose()@X*np.sum((np.linalg.norm(C.transpose()@X,axis=1)**2-D**2))
                        
@pymanopt.function.numpy(manifold_)
def euclidean_hessian(X, U):
    return 2*C@C.transpose()@U

problem = Problem(manifold=manifold_, cost=cost,euclidean_gradient=euclidean_gradient, 
                  euclidean_hessian=euclidean_hessian)

optimizer = optimizers.TrustRegions(max_iterations=900)

h = manifold_.random_point()
Y_init_ =  h 
start = time.time()
Y_star = optimizer.run(problem,initial_point=Y_init_).point
end = time.time()
end-start

(15, 3)
(15, 3)
(15, 3)
Optimizing...
                                            f: +4.645691e+12   |grad|: 1.137892e+09
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc TR+   k:     1     num_inner:     0     f: +4.643643e+12   |grad|: 1.516346e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc TR+   k:     2     num_inner:     0     f: +4.638108e+12   |grad|: 2.282124e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc TR+   k:     3     num_inner:     0     f: +4.623598e+12   |grad|: 3.823780e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc TR-   k:     4     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
REJ TR-   k:     5     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
REJ TR-   k:     6     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded tru

(15, 3)
(15, 3)
(15, 3)
acc       k:   290     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   291     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   292     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   293     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   294     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   295     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   296     num_inner:     0 

(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   604     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   605     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   606     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   607     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   608     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   609     num_inner:     0     f: +4.613566e+12   |grad|: 6.839339e+09   exceeded trust region
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
(15, 3)
acc       k:   610     num_inner

0.5764138698577881

In [46]:
C.shape

(15, 41)

In [42]:
U,S,V = np.linalg.svd(Y_star,full_matrices=False)
S

array([1.41408403e+00, 5.66460988e-09, 2.04875821e-09])

In [190]:
YY = np.diag(S)[:3,:]@V
YY = YY/ np.linalg.norm(YY,axis=0)
position

array([630.,   0., 585.])

In [193]:
direction_to_position(Y_star,C,D)

array([[ 2.60434843e-05,  4.97933347e+01,  4.54147023e+00],
       [ 2.60712579e-05, -4.54143646e+00,  4.97933380e+01],
       [ 5.00000069e+01, -7.67038961e-15,  6.71854709e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.07776010e-05, -2.73173641e+01, -4.87725691e+01],
       [-6.69256602e-06, -2.26271622e+01,  5.11176292e+01],
       [ 7.20359175e+01,  5.27421211e+02, -7.48027822e+01],
       [ 6.94769051e+01,  5.32435411e+02,  2.50388948e+01],
       [ 2.79688106e+02,  5.30627118e+02, -1.95621076e+01],
       [ 3.29300186e+02,  5.36774477e+02, -1.85989112e+01],
       [ 5.80454054e+02,  6.08147130e+02, -4.37804456e+01],
       [ 5.88716052e+02,  5.31485842e+02,  1.98965738e+01],
       [ 5.84997990e+02,  5.95399977e+02,  1.88046547e+01],
       [ 5.85045460e+02,  6.27350969e+02,  5.72645639e+01],
       [ 5.84957192e+02,  6.28078619e+02,  5.79446958e+01]])

In [192]:
def direction_to_position(Y,C,D):
    X = np.linalg.pinv(C@C.transpose())@C@D@Y_star.transpose()
    X = X - X[3,:]
    
    if X[0,0]<0:
        X = -X
    
    v = X[2,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    
    return X@R.transpose()

In [450]:

v = np.array([ 4.99999990e+01, -1.75732342e-02, -7.92915769e-03])

R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))

In [110]:
v = R@Y_star[:,4] 

R1 = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/10)

In [158]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [54]:
X = np.linalg.pinv(C@C.transpose())@C@D@Y_star.transpose()
X

array([[-192.93037171,   91.86004079, -314.48272432,   23.69251499],
       [-220.73226786,   33.82940223, -340.35383563,   37.48321155],
       [-150.85343109,   39.1406749 , -334.77232541,   29.89712156],
       [-186.86418545,   66.24897017, -356.39589818,   30.8056396 ],
       [-188.31010643,   57.63109134, -362.48582007,   85.68326031],
       [-206.19111078,   34.49642711, -365.72338839,   -9.89183372],
       [-184.47966125,  223.50379141,  169.12011704,   26.79175869],
       [-200.43854953,  198.80969644,  166.3533098 ,  -68.74813261],
       [ 165.80946339,  -79.92695645,  227.54803638,   -7.31786382],
       [ 202.62642457, -113.12617493,  233.65111391,   -5.0662883 ],
       [ 181.487853  , -140.25868703,  226.71810499,  -40.6883725 ],
       [ 223.63433024,  -86.14002063,  240.50725253,   30.75839607],
       [ 229.66117739, -110.84740953,  253.45354491,  -26.56152468],
       [ 263.62788206, -107.94362046,  278.12756931,  -53.56194194],
       [ 263.95255345, -107.277225

In [55]:
X = - X + X[3,:]
X

array([[ 6.06618626e+00, -2.56110706e+01, -4.19131739e+01,
         7.11312461e+00],
       [ 3.38680824e+01,  3.24195679e+01, -1.60420626e+01,
        -6.67757195e+00],
       [-3.60107544e+01,  2.71082953e+01, -2.16235728e+01,
         9.08518041e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.44592098e+00,  8.61787883e+00,  6.08992189e+00,
        -5.48776207e+01],
       [ 1.93269253e+01,  3.17525431e+01,  9.32749021e+00,
         4.06974733e+01],
       [-2.38452420e+00, -1.57254821e+02, -5.25516015e+02,
         4.01388091e+00],
       [ 1.35743641e+01, -1.32560726e+02, -5.22749208e+02,
         9.95537722e+01],
       [-3.52673649e+02,  1.46175927e+02, -5.83943935e+02,
         3.81235034e+01],
       [-3.89490610e+02,  1.79375145e+02, -5.90047012e+02,
         3.58719279e+01],
       [-3.68352038e+02,  2.06507657e+02, -5.83114003e+02,
         7.14940121e+01],
       [-4.10498516e+02,  1.52388991e+02, -5.96903151e+02,
      

In [233]:
X[-1,:]-position

array([-1.07039074e+01, -4.44089210e-16, -6.63956294e-01])

In [67]:
R@X[-1,:]

array([620.0773281 ,  72.70829173, 987.47895214])

In [52]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_origin', 'a1_plus'), ('a1_origin', 'a2_plus'), ('a1_origin', ' a2_minus'), ('a1_plus', 'a2_plus'), ('a1_plus', ' a2_minus'), ('a2_plus', ' a2_minus'), ('a2_plus', 'a3_plus'), ('a2_plus', 'a3_minus'), (' a2_minus', 'a3_plus'), (' a2_minus', 'a3_minus'), ('a3_plus', 'a3_minus'), ('a3_plus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_minus', 'a4_minus'), ('a4_origin', 'a4_minus'), ('a4_origin', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_minus', 'a5_minus'), ('a5_plus', 'a5_minus'), ('a5_plus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_minus', 'a6_minus'), ('a6_origin', 'a6_minus')])